In [2]:
import sys
path= 'C:/Users/clement.laplace/Documents/git/NLP-Football'
sys.path.append(path)

In [12]:
import tweepy
import csv
import io
import itertools
import dataset
from utils import *
from textblob import TextBlob
from sqlalchemy.exc import ProgrammingError
import json


In [4]:
tweet_dict = load('EPL_tweet_dict',path+'/Data/Dictionnaries/')
search_terms=list(itertools.chain.from_iterable(tweet_dict.values()))

In [13]:
consumer_key = "JMa27mVbSzGBtBmHspqQ4ln7p"
consumer_secret = "IgkubFPCyPSbk2NGaoca6ULuqoOuFfFhVOnW8wSOclud7g8VSe"
access_key = "1214114293546807297-xkXbFVM3jZHqzdtKnLm303GJJa5qpO"
access_secret = "TCcGYTxEdR08OHFngGw6j8ExAmSsEyB8H6Z4UC3ywf9YZ"
#authorize twitter, initialize tweepy
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)

In [18]:
db = dataset.connect("sqlite:///tweets.db")

class MyStreamListener(tweepy.StreamListener):

    def on_status(self, status):
        if status.retweeted:
            return

        description = status.user.description
        loc = status.user.location
        text = status.text
        coords = status.coordinates
        geo = status.geo
        name = status.user.screen_name
        user_created = status.user.created_at
        followers = status.user.followers_count
        id_str = status.id_str
        created = status.created_at
        retweets = status.retweet_count
        bg_color = status.user.profile_background_color
        blob = TextBlob(text)
        sent = blob.sentiment

        if geo is not None:
            geo = json.dumps(geo)

        if coords is not None:
            coords = json.dumps(coords)

        table = db['EPL_Tweets']
        try:
            table.insert(dict(
                user_description=description,
                user_location=loc,
                coordinates=coords,
                text=text,
                geo=geo,
                user_name=name,
                user_created=user_created,
                user_followers=followers,
                id_str=id_str,
                created=created,
                retweet_count=retweets,
                user_bg_color=bg_color,
                polarity=sent.polarity,
                subjectivity=sent.subjectivity,
            ))
        except ProgrammingError as err:
            print(err)

    def on_error(self, status_code):
        if status_code == 420:
            #returning False in on_data disconnects the stream
            return False

In [19]:
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener=myStreamListener)
myStream.filter(track=search_terms, is_async=True, languages= ["en"])